In [ ]:
import xarray as xr
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.util import add_cyclic_point
import intake, intake_esm
import os
import sys
from pathlib import Path
from glob import glob
import itertools
import time
import seaborn as sns
import math
import scipy.stats as st
import requests

import warnings
warnings.filterwarnings('ignore')

%run functions.ipynb

In [ ]:
'''
Les fichiers sont stockés dans deux catalogues :
    - PMIPCat.json : Modèles Paleo
    - CMIPCat.json : Modèles Actuel (avec piControl notamment)
'''

PMIP = intake.open_esm_datastore("PMIPCat.json")

In [ ]:
CMIP = intake.open_esm_datastore('CMIPCat.json')

In [ ]:
ls /bdd/CMIP6/PMIP/NCC/

# MAPS ANOMALIES

In [ ]:
'''
Import de données terrestre pouvant être utilisée comme
un masque via la fonction 'mask'.

Les étapes :
    - Définition de la résolution pour le futur regrillage
    - import des données via le catalogue PMIPCat
    - Regridde pour être comparé avec d'autres futures données
'''

res_lat = 1.25
res_lon = 1.75

land = PMIP.search(
    institution_id = 'AWI',
    source_id = 'AWI-ESM-1-1-LR',
    table_id = 'Lmon',
    experiment_id = 'lig127k',
    member_id = "r1i1p1f1", 
    latest = True,
    variable_id = 'gpp'
    )

land_ds = xr.open_mfdataset(
    list(land.df["path"]), 
    chunks = 200, 
    use_cftime=True, 
    decode_cf=True
    )
land_ds = land_ds.mean('time')
with ProgressBar():
    land_ds = land_ds.compute()

land_ds = regrid(
    land_ds, 
    res_lat, 
    res_lon, 
    -90, 90, 0, 360
    )

## PMIP3

In [ ]:
dir_pmip3, pmip3 = PMIP3_dic()
var_def = var_dic()
period = {
    'ds_pi':'piControl',
    'ds_lgm': 'lgm'
    }
var = 'hurs'
levels = np.arange(-10, 10, 0.1)

 
for model in pmip3:
    file_dir, filename = read_model(
        model, 
        'lgm',
        var_def, 
        var, 
        dir_pmip3)
    if os.path.exists(file_dir) :
        for k, v in period.items() :
            file_dir, filename = read_model(
                model, 
                v,
                var_def, 
                var, 
                dir_pmip3
                ) 
            
            globals()[k] = xr.open_mfdataset(filename)[var]
            #globals()[k] = zonal_reduction(globals()[k], lon_edc, lat_edc , 5, 2.5)
            globals()[k] = globals()[k].mean(['time']).where(
                (globals()[k].lat < 20) & (globals()[k].lat > -80) & (globals()[k].lon > 0) & (globals()[k].lon < 130), 
                drop = True
                )
        
        ds_anom = ds_lgm - ds_pi
    
        lon = ds_anom.coords['lon']
        lon_idx = ds_anom.dims.index('lon')
        wrap_data, wrap_lon = add_cyclic_point(
            ds_anom.values, 
            coord = lon,
            axis = lon_idx
            )

        plot_seasons(
            wrap_lon,
            ds_anom,
            wrap_data,
            var,
            'Anomalie of Relative Humidity [%]',
            'Difference of mean Surface Relative Humidity of LGM - piControl\nsimulated by {} - {} [PMIP3]'.format(model, pmip3[model]['institute']), 
            levels,
            inst = pmip3[model]['institute'],
            source = model, 
            date = v,
            anomalie = 'anomalie',
            pmip = 'PMIP3',
            mask_land = True
            )


### For unofficial lgm simulations PMIP4

In [ ]:
'''
Il est nécessaire de faire la relation entre les noms des fichiers PMIP
non officiels et ceux notés dans WRCP
'''

name_dic = {
    'AWI-ESM-1-1-LR' : 'AWIESM1',
    'CESM2' : 'CESM2-1',
    'HadGEM3-GC31-LL' : 'HadCM3-ICE6GC',
    'INM-CM4-8' : 'INM-CM4-8',
    'IPSL-CM6A-LR' : 'IPSLCM5A2',
    'MIROC-ES2L' : 'MIROC-ES2L',
    'MPI-ESM1-2-LR' : 'MPI-ESM1-2'
    }

In [ ]:
'''
Script pour plot l'anomalie d'une variable entre
les données LGM OFFICIEUSES et le piControl de WRCP.

Plusieurs étapes :
    - spécification des variables
    - import des données LGM via le catalogue officieux.
      Durant l'import, les données sont :
          . moyennées dans le temps
          . Regrillées
          . le 360° de lon est estimé pour supprimer une bande blance
            durant le futur plot
          . Ouverture du CMIP (r1i1p1f1) s'il n'existe pas il va voir
            un autre forçage (r1i1p1f2)
          . changement de l'unité (en mm.mois-1)
          . calcul de l'anomalie
          . plot (voir section : functions)
'''

i = 'unnamed'
d = 'lgm'
var = 'hurs'
realm = 'Amon'
res_lat = 1.25
res_lon = 1.75
levels = np.arange(-10, 10, 0.1)


for s in name_dic.keys() :
    lgm_ds = xr.open_dataset(
        '/data/bchaigneau/PMIP4/PMIP4_{}_{}_{}_lgm_monClim.nc'.format(var, realm, name_dic.get(s)), 
        chunks = 500
        )
    lgm_ds = lgm_ds.mean('time')
    lgm_ds = regrid(
        lgm_ds,
        res_lat, 
        res_lon, 
        -90, 90, 0, 360
        )
    print('Computing lgm for : {} - {}'.format(s, var))
    with ProgressBar() :
        lgm_ds.compute()
    data = lgm_ds[var]
    lon = lgm_ds.coords['lon']
    lon_idx = data.dims.index('lon')
    wrap_data, wrap_lon = add_cyclic_point(
        data.values, 
        coord = lon, 
        axis = lon_idx
        )

    X = CMIP.search(
        source_id = '{}'.format(s),
        variable_id = var,
        table_id = realm,
        experiment_id = 'piControl',
        latest = True,
        member_id = 'r1i1p1f1'
        )
    if len(X) == 0 :
        X = CMIP.search(
            source_id = '{}'.format(s),
            variable_id = var,
            table_id = realm,
            experiment_id = 'piControl',
            latest = True,
            member_id = 'r1i1p1f2'
            )
        
    institute_ds = xr.open_mfdataset(
        list(X.df["path"]),
        chunks = 500, 
        use_cftime=True,
        decode_cf=True
        )
    pi_ds = institute_ds.mean('time')
    pi_ds = regrid(
        pi_ds,
        res_lat,
        res_lon,
        -90, 90, 0, 360
        )
    print('Computing piControl for : {} - {}'.format(s, var))
    with ProgressBar() :
        pi_ds.compute()
    data2 = pi_ds[var] * 86400 * 365.25/12
    lon2 = pi_ds.coords['lon']
    lon_idx2 = data2.dims.index('lon')
    wrap_data2, wrap_lon = add_cyclic_point(
        data2.values, 
        coord = lon2, 
        axis = lon_idx2
        )

    anomalie = wrap_data - wrap_data2
    plot_seasons(
        wrap_lon, 
        lgm_ds, 
        anomalie, 
        var, 
        'Anomalie of Relative Humidity [%]', 
        'Difference of mean Surface Relative Humidity of\n {} - piControl simulated by {} - {}'.format(d, i, s), 
        levels,
        inst = i,
        source = s,
        date = d,
        anomalie = 'anomalie',
        pmip = 'PMIP3',
        mask_land = True
        )

### For official simulations

In [ ]:
'''
Script pour plot l'anomalie d'une variable entre
les données anciennes OFFICIELES et le piControl de WRCP.

Plusieurs étapes :
    - spécification des variables
    - import des données anciennes via le catalogue officiel.
    - Si le chemin existe pour faire l'anomalie alors on run.
      Durant l'import, les données sont :
          . moyennées dans le temps
          . Regrillées
          . Certaines coordonnées sont sélectionnées
          . Ouverture du CMIP (r1i1p1f1) 
          . Calcul de l'anomalie puis plot (voir : functions)
    - Si les données n'existent pas alors il essaye pour les
      forçages r1i1p1f2.
'''


# np.arange(0, 1.2e-7, 4e-9)  lat: -40/40  lon: <180 >240  Greens
var = 'hurs'
realm = 'Amon'
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
date = ['lgm', 'lig127k']
grid = ['gn', 'gr1', 'gr']
res_lat = 1.25
res_lon = 1.75
levels = np.arange(-10, 10, 0.1)

for i, s, d, g in itertools.product(inst, source, date, grid):
        if (os.path.exists('/bdd/CMIP6/PMIP/{}/{}/{}/r1i1p1f1/{}/{}/{}/latest'.format(i, s, d, realm, var, g))
           ) and (os.path.exists('/bdd/CMIP6/CMIP/{}/{}/piControl/r1i1p1f1/{}/{}/{}/latest'.format(i, s, realm, var, g))) :
            X = PMIP.search(
                institution_id= '{}'.format(i), 
                variable_id = var,
                source_id = '{}'.format(s), 
                table_id = realm,
                experiment_id= '{}'.format(d),
                latest = True,
                member_id = "r1i1p1f1",
                grid_label = '{}'.format(g)
                )
            institute_ds = xr.open_mfdataset(
                list(X.df["path"]),
                chunks = 500, 
                use_cftime=True,
                decode_cf=True
                )
            dsMoy = institute_ds.mean("time")
            dsMoy = regrid(
                dsMoy, 
                res_lat, 
                res_lon,
                -90, 90, 0, 360
                )
            # dsMoy = mask(dsMoy, land_ds, 'hurs')

            print('Computing {} for : {} - {} :'.format(d, s, var))
            with ProgressBar():
                dsMoy = dsMoy.compute()
              
            dsMoy = dsMoy.where((
                (dsMoy.coords['lat'] < 20) 
                & (dsMoy.coords['lat'] > -80)),
                drop = True
                )
            dsMoy = dsMoy.where((
                (dsMoy.coords['lon'] < 130) 
                & (dsMoy.coords['lon'] > 0)),
                drop = True
                )  
            
            data = dsMoy[var]#* 86400 * 365.25/12
            lon = dsMoy.coords['lon']
            lon_idx = data.dims.index('lon')
            wrap_data, wrap_lon = add_cyclic_point(
                data.values, 
                coord = lon, 
                axis = lon_idx
                )
  
            X = CMIP.search(
                source_id = '{}'.format(s), 
                variable_id = var, 
                table_id = realm,
                experiment_id = 'piControl',
                latest = True,
                member_id = 'r1i1p1f1'
                )
            institute_ds = xr.open_mfdataset(
                list(X.df["path"]),
                chunks = 500,
                use_cftime=True,
                decode_cf=True
                )
            pi_ds = institute_ds.mean('time')
            pi_ds = regrid(
                pi_ds,
                res_lat,
                res_lon, 
                -90, 90, 0, 360
                )
            
            # pi_ds = mask(pi_ds, land_ds, 'hurs')
            
            print('Computing piControl for : {} - {}'.format(s, var))
            with ProgressBar() :
                pi_ds.compute()
            
            pi_ds = pi_ds.where((
                (pi_ds.coords['lat'] < 20) 
                & (pi_ds.coords['lat'] > -80)),
                drop = True
                )
            pi_ds = pi_ds.where((
                (pi_ds.coords['lon'] < 130) 
                & (pi_ds.coords['lon'] > 0)),
                drop = True
                )  
            
            data2 = pi_ds[var]#*  86400 * 365.25/12
            lon2 = pi_ds.coords['lon']
            lon_idx2 = data2.dims.index('lon')
            wrap_data2, wrap_lon = add_cyclic_point(
                data2.values, 
                coord = lon2,
                axis = lon_idx2
                )

            anomalie = (wrap_data - wrap_data2)
            
            plot_seasons(
                wrap_lon, 
                pi_ds, 
                anomalie, 
                var, 
                'Anomalie of Relative Humidity[%]', 
                'Difference of mean Surface Relative Humidity of {} - piControl\nsimulated by {} - {} [PMIP4]'.format(d, i, s), 
                levels,
                inst = i,
                source = s,
                date = d,
                anomalie = 'anomalie',
                pmip = 'PMIP4',
                mask_land = True
                )
            
            
        elif (os.path.exists('/bdd/CMIP6/PMIP/{}/{}/{}/r1i1p1f2/{}/{}/{}/latest'.format(i, s, d, realm, var, g))
             ) and (os.path.exists('/bdd/CMIP6/CMIP/{}/{}/piControl/r1i1p1f2/{}/{}/{}/latest'.format(i, s, realm, var, g))) :
            X = PMIP.search(
                institution_id= '{}'.format(i), 
                variable_id = var, 
                source_id = '{}'.format(s),
                table_id = realm, 
                experiment_id= '{}'.format(d), 
                latest = True,
                member_id = "r1i1p1f2", 
                grid_label = '{}'.format(g)
                )
            institute_ds = xr.open_mfdataset(
                list(X.df["path"]),
                chunks = 500, 
                use_cftime=True,
                decode_cf=True
                )
            dsMoy = institute_ds.mean("time")
            dsMoy = regrid(
                dsMoy, 
                res_lat, 
                res_lon, 
                -90, 90, 0, 360
                )
            
            # dsMoy = mask(dsMoy, land_ds, 'hurs')
            
            print('Computing {} for : {} - {} :'.format(d, s, var))
            with ProgressBar():
                dsMoy = dsMoy.compute()
            
            dsMoy = dsMoy.where((
                (dsMoy.coords['lat'] < 20) 
                & (dsMoy.coords['lat'] > -80)),
                drop = True
                )
            dsMoy = dsMoy.where((
                (dsMoy.coords['lon'] < 130)
                & (dsMoy.coords['lon'] > 0)),
                drop = True
                ) 
            
            data = dsMoy[var]#* 86400 * 365.25/12
            lon = dsMoy.coords['lon']
            lon_idx = data.dims.index('lon')
            wrap_data, wrap_lon = add_cyclic_point(
                data.values, 
                coord = lon,
                axis = lon_idx
                )
               
            X = CMIP.search(
                source_id = '{}'.format(s), 
                variable_id = var, 
                table_id = realm, 
                experiment_id = 'piControl',
                latest = True,
                member_id = 'r1i1p1f2'
                )
            institute_ds = xr.open_mfdataset(
                list(X.df["path"]),
                chunks = 500, 
                use_cftime=True,
                decode_cf=True
                )
            pi_ds = institute_ds.mean('time')
            pi_ds = regrid(
                pi_ds, 
                res_lat, 
                res_lon, 
                -90, 90, 0, 360
                )
            
            # pi_ds = mask(pi_ds, land_ds, 'hurs')
            
            print('Computing piControl for : {} - {}'.format(s, var))
            with ProgressBar() :
                pi_ds.compute()
            
            pi_ds = pi_ds.where((
                (pi_ds.coords['lat'] < 20) 
                & (pi_ds.coords['lat'] > -80)), 
                drop = True
                )
            pi_ds = pi_ds.where((
                (pi_ds.coords['lon'] < 130) 
                & (pi_ds.coords['lon'] > 0)),
                drop = True
                ) 
            
            data2 = pi_ds[var]#* 86400 * 365.25/12
            lon2 = pi_ds.coords['lon']
            lon_idx2 = data2.dims.index('lon')
            wrap_data2, wrap_lon = add_cyclic_point(
                data2.values, 
                coord = lon2, 
                axis = lon_idx2
                )

            anomalie = (wrap_data - wrap_data2)
            
            plot_seasons(
                wrap_lon, 
                pi_ds, 
                anomalie, 
                var, 
                'Anomalie of Relative Humidity [%]', 
                'Difference of mean Surface Relative Humidity of {} - piControl\nsimulated by {} - {} [PMIP4]'.format(d, i, s), 
                levels,
                inst = i,
                source = s,
                date = d,
                anomalie = 'anomalie',
                pmip = 'PMIP4',
                mask_land = True
                )

# PLOT

## Between models

In [ ]:
# np.arange(0, 1.2e-7, 4e-9)  lat: -40/40  lon: <180 >240  Greens
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
date = ['lgm', 'lig127k']
levels = np.arange(0, 1.2e-7, 4e-9)

for d in date :  
    sns.set_style('ticks')
    sns.set_context("talk")
    sns.set_style(
        'ticks',
        {
            "grid.color":"0.9", 
            "grid.linestyle":":", 
            "axes.grid": True
        })
    fig, ax = plt.subplots(figsize = (5, 12))
    b = 0
    for i, s in itertools.product(inst, source):
           
        if os.path.exists('/home/bchaigneau/Stage_LSCE/plot/gpp/comparision_models_50%/{}/'.format(d)) == False :
            path = os.path.join('/home/bchaigneau/Stage_LSCE/plot/gpp/comparision_models_50%', '{}'.format(d))
            os.makedirs(path)
        
        
        if os.path.exists('/bdd/CMIP6/PMIP/{}/{}/{}/r1i1p1f1/Lmon/gpp/gn/latest'.format(i, s, d)) :
            X = PMIP.search(
                institution_id= '{}'.format(i), 
                variable_id = 'gpp',
                source_id = '{}'.format(s),
                table_id = "Lmon", 
                experiment_id= '{}'.format(d), 
                latest = True, 
                member_id = "r1i1p1f1"
                )   
            institute_ds = xr.open_mfdataset(
                list(X.df["path"]),
                chunks = 1000, 
                use_cftime=True,
                decode_cf=True
                )
            dsMoy = institute_ds.mean("time")
            
            dsMoy = dsMoy.where((
                (dsMoy.coords['lat'] < 40) 
                & (dsMoy.coords['lat'] > -40)),
                drop = True
                )
            dsMoy = dsMoy.where((
                (dsMoy.coords['lon'] < 180) 
                | (dsMoy.coords['lon'] > 240)), 
                drop = True
                )
            
            threshold = 0.5 * dsMoy['gpp'].max()
            dsMoy = dsMoy.where(dsMoy['gpp'] >= threshold)
            dsMoy = dsMoy.mean('lon')
            print('Creation of the df : {} - gpp - {} :'.format(i, d))
            with ProgressBar():
                dsMoy = dsMoy.compute()
                
            dfMoy = pd.DataFrame()
            dfMoy['lat'] = dsMoy['lat']
            dfMoy['gpp'] = dsMoy['gpp']
            dfMoy['institute'] = i
            dfMoy[ 'gpp_avg' ] = dfMoy.gpp.rolling(10).mean() 
            
            length = PMIP.search(
                variable_id = 'gpp', 
                table_id = 'Lmon',
                experiment_id = '{}'.format(d), 
                latest = True, 
                member_id = 'r1i1p1f1',
                grid_label = 'gn'
                )
            
            a = sns.color_palette(
                'crest_r', 
                len(length.df.institution_id.unique())
                )
            plt.plot(
                dfMoy['gpp_avg'], dfMoy['lat'], 
                linewidth = 4, 
                color = a[b],
                label = '{}'.format(i)
                )
            # plt.fill_between(dfMoy['gpp_avg'], dfMoy['lat'], alpha = .1, color = a[b])
            b += 1
    plt.ylim(-32, 32)
    plt.legend(loc='upper right')
    plt.ylabel('Latitude [°N]')
    plt.xlabel('Gross Primary Productivity [kg.m-2.s-1]', labelpad = 20)
    plt.title('Comparision of gross primary productivity\nbetween models ({})'.format(d), fontweight = 'bold', pad = 25, fontsize = 20)
    plt.savefig('/home/bchaigneau/Stage_LSCE/plot/gpp/comparision_models_50%/{}/{}_{}_gpp_{}.jpeg'.format(d, i, s, d), dpi = 150)
    plt.show()

## Between periods

In [ ]:
# np.arange(0, 1.2e-7, 4e-9)  lat: -40/40  lon: <180 >240  Greens
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
date = ['lgm', 'lig127k']
levels = np.arange(0, 1.2e-7, 4e-9)

choice = PMIP.df[
    (PMIP.df['member_id'] == 'r1i1p1f1') 
    & (PMIP.df['table_id'] == 'Lmon') 
    & (PMIP.df['grid_label'] == 'gn')
    & (PMIP.df['latest'] == True) 
    & (PMIP.df['variable_id'] == 'gpp')
    ]
choice = choice[
    (choice['experiment_id'] == 'lig127k') 
    | (choice['experiment_id'] == 'lgm')
    ]
choice = choice.iloc[:,3:10].drop_duplicates(subset = ['institution_id', 'source_id'])

for c in range(len(choice)) :
    ch = choice.iloc[c]
    i = ch['institution_id']
    s = ch['source_id']
    sns.set_style('ticks')
    sns.set_context("talk")
    sns.set_style(
        'ticks',
        {
            "grid.color":"0.9", 
            "grid.linestyle":":", 
            "axes.grid": True
        })
    fig, ax = plt.subplots(figsize = (5, 12))
    b = 0
    length = PMIP.search(
        variable_id = 'gpp', 
        table_id = 'Lmon',
        latest = True,
        member_id = 'r1i1p1f1',
        grid_label = 'gn', 
        experiment_id = ['lgm', 'lig127k']
        )

    for d in date :    
        if os.path.exists('/bdd/CMIP6/PMIP/{}/{}/{}/r1i1p1f1/Lmon/gpp/gn/latest'.format(i, s, d)) :
            if os.path.exists('/home/bchaigneau/Stage_LSCE/plot/gpp/comparision_periods_50%/{}/{}/'.format(i, s)) == False :
                path = os.path.join('/home/bchaigneau/Stage_LSCE/plot/gpp/comparision_periods_50%', '{}'.format(i), '{}'.format(s))
                os.makedirs(path)
                
            X = PMIP.search(
                institution_id= '{}'.format(i), 
                variable_id = 'gpp',
                source_id = '{}'.format(s),
                table_id = "Lmon", 
                experiment_id= '{}'.format(d), 
                latest = True, 
                member_id = "r1i1p1f1"
                )   
            institute_ds = xr.open_mfdataset(
                list(X.df["path"]),
                chunks = 1000,
                use_cftime=True,
                decode_cf=True
                )
            
            dsMoy = institute_ds.mean("time")
            dsMoy = dsMoy.where((
                (dsMoy.coords['lat'] < 40) 
                & (dsMoy.coords['lat'] > -40)),
                drop = True
                )
            dsMoy = dsMoy.where((
                (dsMoy.coords['lon'] < 180) 
                | (dsMoy.coords['lon'] > 240)),
                drop = True
                )
            threshold = 0.5 * dsMoy['gpp'].max()
            dsMoy = dsMoy.where(dsMoy['gpp'] >= threshold)
            dsMoy = dsMoy.mean('lon')
            print('Creation of the df : {} / {} / gpp :'.format(i, s))
            with ProgressBar():
                dsMoy = dsMoy.compute()
            dfMoy = pd.DataFrame()
            dfMoy['lat'] = dsMoy['lat']
            dfMoy['gpp'] = dsMoy['gpp']
            dfMoy['institute'] = i
            dfMoy[ 'gpp_avg' ] = dfMoy.gpp.rolling(10).mean() 
            
            
            a = sns.color_palette(
                'crest_r', 
                len(length.df.experiment_id.unique())
                )
            plt.plot(
                dfMoy['gpp_avg'], dfMoy['lat'], 
                linewidth = 4, 
                color = a[b],
                label = '{}'.format(d)
                )
            # plt.fill_between(dfMoy['gpp_avg'], dfMoy['lat'], alpha = .1, color = a[b])
            b += 1
    plt.ylim(-32, 32)
    plt.legend(loc='upper right')
    plt.ylabel('Latitude [°N]')
    plt.xlabel('Gross Primary Productivity [kg.m-2.s-1]', labelpad = 20)
    plt.title('Comparision of gross primary productivity\nbetween periods \n\n({}-{})'.format(i, s), fontweight = 'bold', pad = 25, fontsize = 20)
    plt.savefig('/home/bchaigneau/Stage_LSCE/plot/gpp/comparision_periods_50%/{}/{}/{}_{}_gpp.jpeg'.format(i, s, i, s), dpi = 150)
    plt.show()

# ------- AUTOMATIC DATA AGGREGATION + SUBPLOTS

In [ ]:
ls /bdd/CMIP6/PMIP/INM/INM-CM4-8/lig127k/*/Lmon/ra

In [ ]:
land = PMIP.search(
    institution_id = 'INM',
    source_id = 'INM-CM4-8',
    table_id = 'Lmon',
    experiment_id = 'lig127k',
    member_id = "r1i1p1f1", 
    latest = True,
    variable_id = 'ra'
    )

land_ds = xr.open_mfdataset(
    list(land.df["path"]), 
    chunks = 200, 
    use_cftime=True,
    decode_cf=True
    )
land_ds = land_ds.mean('time')
with ProgressBar():
    land_ds = land_ds.compute()


#land_ds = land_ds.where((land_ds.coords['lon'] < 150) & (land_ds.coords['lon'] > 20) & (land_ds.coords['lat'] <30) & (land_ds.coords['lat'] > -70), drop = True)

land_ds = land_ds.where(land_ds['ra'] > 0)

In [ ]:
land_ds.ra.plot()

### PMIP3 datas

In [ ]:
dir_pmip3, pmip3 = PMIP3_dic()
var_def = var_dic()
period = {
    'ds_pi':'piControl',
    'ds_lgm': 'lgm'
    }
grid = ['gn', 'gr1', 'gr']
res_lat = 1.25
res_lon = 1.875

var = 'gpp'

df_pmip3 = pd.DataFrame(columns = ['lat', 'institute', 'source', 'period', 'type'])


for model in pmip3:
    file_dir, filename = read_model(
        model, 
        'lgm',
        var_def, 
        var, 
        dir_pmip3
        )
    if os.path.exists(file_dir) :
        for k, v in period.items() :
            file_dir, filename = read_model(
                model, 
                v,
                var_def, 
                var, 
                dir_pmip3
                )
            
            globals()[k] = xr.open_mfdataset(filename, chunks = 200)[var]
            globals()[k] = globals()[k].compute() 
            globals()[k] = globals()[k].mean(['time'])
            #globals()[k] = zonal_reduction(globals()[k], 75, -35 , 55, 35)
            globals()[k] = mask2(globals()[k], land_ds, var)
            globals()[k] = globals()[k].where((globals()[k] > 0) & (globals()[k] < 1e30))
            globals()[k] = globals()[k].mean(['lon']) * 86400 * 365.25/12

        anom_ds = ds_lgm - ds_pi
        
        data = pd.DataFrame()
        data['lat'] = anom_ds['lat']
        data['institute'] = pmip3[model]['institute']
        data['source'] = model
        data['period'] = 'lgm'
        data['type'] = 'pmip3'
        data[var] = anom_ds.values
        df_pmip3 = df_pmip3.append(data)        

### Unofficial datas

In [ ]:
# If needed to download from officious database :
name_dic = {
    'AWI-ESM-1-1-LR' : 'AWIESM1',
    'CESM2' : 'CESM2-1',
    'HadGEM3-GC31-LL' : 'HadCM3-ICE6GC',
    'INM-CM4-8' : 'INM-CM4-8',
    'IPSL-CM6A-LR' : 'IPSLCM5A2',
    'MIROC-ES2L' : 'MIROC-ES2L',
    'MPI-ESM1-2-LR' : 'MPI-ESM1-2'
    }

In [ ]:
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
grid = ['gn', 'gr1', 'gr']
res_lat = 1.25
res_lon = 1.875

var = 'gpp'
df_unoff = pd.DataFrame(columns = ['lat', 'institute', 'source', 'period', 'type'])


    # Unofficial LGM
for s in name_dic.keys() :
    for i in inst :
        for g in grid :
            if os.path.exists('/bdd/CMIP6/CMIP/{}/{}/piControl/r1i1p1f1/Amon/{}/{}/latest'.format(i, s, var, g)) :
                lgm_ds = xr.open_dataset(
                    '/data/bchaigneau/PMIP4/PMIP4_{}_Amon_{}_lgm_monClim.nc'.format(var, name_dic.get(s)), 
                    chunks = 500
                    )
                lgm_ds = lgm_ds.mean('time')
                lgm_ds = regrid(
                    lgm_ds, 
                    res_lat,
                    res_lon,
                    -90, 90, 0, 360
                    )
                #lgm_ds = lgm_ds.where(((lgm_ds.coords['lat'] < 0) & (lgm_ds.coords['lat'] > -70)), drop = True)
                #lgm_ds = lgm_ds.where(((lgm_ds.coords['lon'] < 130) & (lgm_ds.coords['lon'] > 20)), drop = True) 
                print('Computing lgm for : {} - {}'.format(s, var))
                with ProgressBar() :
                    lgm_ds.compute()
                lgm_ds = mask2(lgm_ds, land_ds, var)
                lgm_ds = lgm_ds.mean('lon') #/ 12

                # piControl associé
                Y = CMIP.search(
                    institution_id = i,
                    variable_id = var,
                    source_id = s,
                    experiment_id = 'piControl',
                    table_id = 'Amon',
                    latest = True,
                    member_id = 'r1i1p1f1'
                    )
                pi_ds = xr.open_mfdataset(
                    list(Y.df['path']),
                    chunks = 200,
                    use_cftime = True, 
                    decode_cf = True
                    )
                pi_ds = pi_ds.mean('time')
                print('Computing piControl for : {} - {} - {}'.format(i, s, var))
                with ProgressBar():
                    pi_ds = pi_ds.compute()   
                pi_ds = regrid(
                    pi_ds, 
                    res_lat, 
                    res_lon,
                    -90, 90, 0, 360
                    )
                pi_ds = pi_ds.where(((pi_ds.coords['lat'] < 0) & (pi_ds.coords['lat'] > -70)), drop = True)
                pi_ds = pi_ds.where(((pi_ds.coords['lon'] < 130) & (pi_ds.coords['lon'] > 20)), drop = True) 
                pi_ds = mask2(pi_ds, land_ds, var)
                pi_ds = pi_ds.mean('lon') #/12


                anom_ds = lgm_ds - pi_ds

                data = pd.DataFrame()
                data['lat'] = anom_ds['lat']
                data['institute'] = i
                data['source'] = s
                data['period'] = 'lgm'
                data['type'] = 'unofficial'
                data[var] = anom_ds[var]
                df_unoff = df_unoff.append(data)


### Official datas

In [ ]:
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
grid = ['gn', 'gr1', 'gr']
res_lat = 1.25
res_lon = 1.875

var = 'gpp'
table_id = 'Lmon'
df_off = pd.DataFrame(columns = ['lat', 'institute', 'source', 'period'])


for s in source :
    for i in inst :
        for g in grid :
        # Pre-industrial Control
            if (os.path.exists(
                '/bdd/CMIP6/CMIP/{}/{}/piControl/r1i1p1f1/{}/{}/{}/latest'.format(i, s, table_id, var, g))) & (
                (os.path.exists('/bdd/CMIP6/PMIP/{}/{}/lgm/r1i1p1f1/{}/{}/{}/latest'.format(i, s, table_id, var, g))) | (os.path.exists('/bdd/CMIP6/PMIP/{}/{}/lig127k/r1i1p1f1/{}/{}/{}/latest'.format(i, s, table_id, var, g)))
                ) :
                Y = CMIP.search(
                    institution_id = '{}'.format(i),
                    variable_id = var,
                    source_id = '{}'.format(s),
                    table_id = table_id,
                    experiment_id = 'piControl',
                    latest = True,
                    member_id = 'r1i1p1f1'
                    )
                pi_ds = xr.open_mfdataset(
                    list(Y.df['path']),
                    chunks = 200,
                    use_cftime = True, 
                    decode_cf = True
                    )[var]
                pi_ds = pi_ds.mean('time')
                print('Computing piControl for : {} - {} - {}'.format(i, s, var))
                with ProgressBar():
                    pi_ds = pi_ds.compute()   
                pi_ds = regrid(
                    pi_ds, 
                    res_lat,
                    res_lon,
                    -90, 90, 0, 360
                    )
                
                #pi_ds = pi_ds.where(((pi_ds.coords['lat'] < 0) & (pi_ds.coords['lat'] > -70)), drop = True)
                #pi_ds = pi_ds.where(((pi_ds.coords['lon'] < 130) & (pi_ds.coords['lon'] > 20)), drop = True) 
                pi_ds = mask2(pi_ds, land_ds, var)
                pi_ds = pi_ds.where((pi_ds > 0) & (pi_ds < 1e30))
                pi_ds = pi_ds.mean('lon') * 86400 * 365.25/12

    
        # Official LGM
                if os.path.exists('/bdd/CMIP6/PMIP/{}/{}/lgm/r1i1p1f1/{}/{}/{}/latest'.format(i, s, table_id, var, g)) :
                    #if s not in name_dic.keys() : 
                    Z = PMIP.search(
                        institution_id = '{}'.format(i),
                        variable_id = var, 
                        source_id = '{}'.format(s),
                        table_id = table_id,
                        experiment_id = 'lgm',
                        latest = True, 
                        member_id = 'r1i1p1f1',
                        grid_label = g
                        )
                    oflgm_ds = xr.open_mfdataset(
                        list(Z.df['path']),
                        chunks = 200, 
                        use_cftime = True,
                        decode_cf = True
                        )[var]
                    oflgm_ds = oflgm_ds.mean('time')
                    print('Computing official lgm for : {} - {} - {}'.format(i, s, var))
                    with ProgressBar():
                        oflgm_ds = oflgm_ds.compute() 
                    oflgm_ds = regrid(
                        oflgm_ds,
                        res_lat, 
                        res_lon,
                        -90, 90, 0, 360
                        )
                    #oflgm_ds = oflgm_ds.where(((oflgm_ds.coords['lat'] < 0) & (oflgm_ds.coords['lat'] > -70)), drop = True)
                    #oflgm_ds = oflgm_ds.where(((oflgm_ds.coords['lon'] < 130) & (oflgm_ds.coords['lon'] > 20)), drop = True) 
                    oflgm_ds = mask2(oflgm_ds, land_ds, var)
                    oflgm_ds = oflgm_ds.where((oflgm_ds > 0) & (oflgm_ds < 1e30))
                    oflgm_ds = oflgm_ds.mean('lon') * 86400 * 365.25/12

                    anom_ds = oflgm_ds - pi_ds

                    data = pd.DataFrame()
                    data['lat'] = anom_ds['lat']
                    data['institute'] = i
                    data['source'] = s
                    data['period'] = 'lgm'
                    data['type'] = 'official'
                    data[var] = anom_ds
                    df_off = df_off.append(data)      

        # Lig127k
                if os.path.exists('/bdd/CMIP6/PMIP/{}/{}/lig127k/r1i1p1f1/{}/{}/{}/latest'.format(i, s, table_id, var, g)) :
                    X = PMIP.search(
                        institution_id= '{}'.format(i), 
                        variable_id = var,
                        source_id = '{}'.format(s), 
                        table_id = table_id,
                        experiment_id= 'lig127k',
                        latest = True,
                        member_id = "r1i1p1f1",
                        grid_label = g
                        )   
                    institute_ds = xr.open_mfdataset(
                        list(X.df["path"]),
                        chunks = 200,
                        use_cftime=True,
                        decode_cf=True
                        )[var]
                    if 'longitude' in list(institute_ds.coords):
                        institute_ds = institute_ds.rename({
                            'longitude': 'lon',
                            'latitude': 'lat'
                            })
                    # dsMoy = dsMoy.fillna(0)
                    dsMoy = institute_ds.mean("time")
                    print('Computing lig127k for : {} - {} - {}'.format(i, s, var))
                    with ProgressBar() :
                        dsMoy = dsMoy.compute()  
                    dsMoy = regrid(
                        dsMoy, 
                        res_lat,
                        res_lon,
                        -90, 90, 0, 360
                        )
                    #land = regrid(land_ds, res_lat, res_lon, -90, 90, 0, 360)
                    #dsMoy = dsMoy.where(((dsMoy.coords['lat'] < 0) & (dsMoy.coords['lat'] > -70)), drop = True)
                    #dsMoy = dsMoy.where(((dsMoy.coords['lon'] < 130) & (dsMoy.coords['lon'] > 20)), drop = True) 
                    dsMoy = mask2(dsMoy, land_ds, var)
                    #dsMoy = mask(dsMoy, land, 'gpp')
                    dsMoy = dsMoy.where((dsMoy > 0) & (dsMoy < 1e30))
                    dsMoy = dsMoy.mean('lon') * 86400 * 365.25/12

                    anom_ds = dsMoy - pi_ds

                    #dsMoy = dsMoy.where(dsMoy.gpp.notnull(), drop = True)    
                    data = pd.DataFrame()
                    data['lat'] = anom_ds['lat']
                    data['institute'] = i
                    data['source'] = s
                    data['period'] = 'lig127k'
                    data['type'] = 'official'
                    data[var] = anom_ds
                    df_off = df_off.append(data)
                    
# df[var] = df[var].fillna(0) 
#df = df.reset_index()

In [ ]:
df = df_off.append([df_pmip3])
df = df.reset_index(drop = True)
df = df.dropna()
df = df[df['gpp'] > -1e30]

In [ ]:
var = 'gpp'

sns.set_context("talk")
sns.set_style('ticks')
sns.set_context("talk")
sns.set_style(
    'ticks', {
        "grid.color":"0.9", 
        "grid.linestyle":":", 
        "axes.grid": True
        })
my_pal = {
    'lgm' : '#88CCEE', 
    'lig127k' : '#CC6677'
    }

fig, axes = plt.subplots(
    figsize = (15, 12)
    )

x = df[['source', 'period', 'type']].drop_duplicates()

for idx, v in x.iterrows():
    df_inter = df[(df['period'] == v['period']) & (df['type'] == v['type']) & (df['source'] == v['source'])]
    sns.lineplot(
        data = df_inter, 
        x = var, y = 'lat', 
        sort = False, 
        color = my_pal[df_inter['period'].unique()[0]],
        lw = 2,
        #alpha = .8
        )

plt.xlabel('\n Gross Primary Productivity [kg.m-2.month-1]', fontweight = 'bold', size = 22)
plt.ylabel('Latitude [°N]\n', fontweight = 'bold', size = 22)

plt.title(
    'Distribution of Mean Gross Primary Productivity Anomalie across\nthe Latitude gradient simulated by all of the PMIP models\n', 
    fontweight = 'bold',
    size = 25,
    )

marker = [plt.Line2D([0,0], [0,0], color = color) for color in my_pal.values()]
plt.legend(
    marker, my_pal.keys(),
    loc = 'upper right',
    bbox_to_anchor = (1.2, 1),
    fontsize = 20
    )

plt.text(-0.06, -95, 
         'For one latitude, values are meaned by all longitudes at global level,\nAnomalie value is corresponding to the studied period (LGM or LIG127) minus piControl',
         fontstyle = 'italic'
         )

plt.savefig(
    '/home/bchaigneau/Stage_LSCE/plot/{}/{}_models_bylatitude.jpeg'.format(var, var), 
    dpi = 150, 
    bbox_inches = 'tight'
    )


## Subplots

In [ ]:
df = df[df['gpp'] != 0]

# Variable of interest
var = 'tas'

# Get the size you want for your subplots
num_rows = 2
num_cols = int(np.ceil(len(df.source.unique())/num_rows))
rang = df['{}'.format(var)].max() - df['{}'.format(var)].min()
xmin = df['{}'.format(var)].min() - 1/10*(rang)
xmax = df['{}'.format(var)].max() + 1/10*(rang)
rangy =df['lat'].max() - df['lat'].min()
ymin = df['lat'].min() - 1/20*rangy
ymax = df['lat'].max() + 1/20*rangy


sns.set_context("talk")
sns.set_style('ticks')
sns.set_context("talk")
sns.set_style(
    'ticks', {
        "grid.color":"0.9", 
        "grid.linestyle":":", 
        "axes.grid": True
        })
my_pal = {
    'lgm' : '#88CCEE', 
    'lig127k' : '#CC6677',
    'piControl' : '#999933'
    }

fig, axes = plt.subplots(
    num_rows, 
    num_cols,
    figsize = (30, 17)
    )


for index, s in enumerate(df.source.unique()) :
    inter = df[df['source'] == s]
    if index < num_cols :
        i = 0
    if index > num_cols - 1 :
        i = 1
        index = index - num_cols     

    ax = sns.lineplot(
        data = inter, 
        hue = 'period', 
        x = '{}'.format(var), y = 'lat', 
        palette = my_pal,
        ax = axes[i, index], 
        sort = False
        )
    sns.despine(
        ax = axes[i, index],
        offset = 10
        )
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.locator_params(axis='x', nbins=5)
    # ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    if index != 0 :
        ax.axes.yaxis.set_ticklabels([])
    ax.set(xlabel = None, ylabel = None)
    ax.get_legend().remove()
    ax.text(
        xmin + rang/2, 
        ymax + 1/25*ymax,
        s,
        horizontalalignment = 'center',
        verticalalignment = 'top',
        size = 14, 
        bbox = dict(
            facecolor="white",
            edgecolor="black",
            boxstyle="round", 
            pad=0.2
        ))
    
fig.delaxes(axes[1,6])
        
pi_patch = mpatches.Patch(
    color='#999933',
    label='piControl'
    )
lig_patch = mpatches.Patch(
    color='#CC6677',
    label='lig127k'
    )
lgm_patch = mpatches.Patch(
    color='#88CCEE',
    label='lgm'
    )

ax.legend(
    handles = [
        pi_patch, 
        lig_patch, 
        lgm_patch
        ], 
    loc = 'upper right', 
    bbox_to_anchor = (2.05, 0.8), 
    shadow = True,
    title = 'Periods'
    )
fig.text(
    0.07, 0.5,
    'Latitude [°N]',
    ha='center', va='center', 
    rotation='vertical',
    fontweight = 'bold',
    fontsize = 25
    )
fig.text(
    0.5, 0.05,
    'TAS [°K]',
    ha='center', va='center',
    fontweight = 'bold', 
    fontsize = 25
    )
fig.text(
    0.01, 0.99,
    'Distribution of the mean TAS\nby latitude simulated by the different models', 
    ha='left', va='center',
    fontweight = 'bold', 
    fontsize = 40
    )
plt.savefig(
    '/home/bchaigneau/Stage_LSCE/plot/{}/{}_comp_models_periods_generalized.jpeg'.format(var, var), 
    dpi = 250, 
    bbox_inches = 'tight'
    )